# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# Cleaning

Useful Columns:

- video_id - No changes, used to uniquely identify vids

- publishedAt - Convert to datetime, use to determine video age, filter out < 3 month old

- channel_id - No changes

- title - NLP

- description - NLP

- thumbnail_url - Keep in case we want to analyze thumbnail traits

- channel_name - No changes (not in model)

- tags - NLP

- duration - Convert to time delta or float value to represent length in seconds

- view - use with likes for likes-to-views ratio

- like - use with views for likes-to-views ratio

- comment - comment count, leave as integer count

- has_paid_placement - A flag that specifies whether the video has paid product placement or not. It may be useful for determining if enabling this flag affects video success.

- sponsored - A flag that specifies whether the video has paid product or sponsorship-related keywords in the description using regex. It may be useful for determining if sponsored content impacts video success.

- collection_time - The time at which this data was collected. Useful in conjunction with publish time to obtain a given video's "age"

- short - decide whether to filter out shorts or not

- description_channel - Description of the channel itself. May be used to quantify what a channel is about or what they typically upload, and whether or not that can be used as an indicator for video success

- country - Will be used to remove channels that are not based in the US

- view_count - The total numbers of views across all videos for the given channel. May be useful for contextualizing a specific video's views against other videos published by the same channel. Will not be used as input to the model

- subscriber_count  - The total number of subscribers or channel size at the time of data collection. It may be useful in determining whether channel size correlates to video success.

- video_count - The total number of videos published by this channel. Will not be used as input to the model

- branding_keywords - Keywords associated with the channel

- rank - keep for reference. Will not be used as input for the model.

# Derived Attributes

- Title and Description - VADER through nltk (2)

- Average Engagement from past X videos (1)

- Standard deviation of the time between video uploads (1)

- Hours since last upload (1)

- Length of title, description (2)

- Tag correlation (title, comment, description) (3) -> is this different from tags?

- TFIDF for description titles across channels? (2)

- Top 10 comments by likes for each video. Average length, sentiment, readability (Flesch, Kincaid) (4)

- Has comments turned off, video is private

# Cleaning Steps

1. Subset columns to useful ones listed above

2. Change column names to informative names

3. Convert duration to integer seconds

4. Filter out videos younger than 3 months

5. Filter out shorts

Archive private videos

6. Create likes / views column

7. Filter out channels with non-US countries

8. Filter out livestreams (duration <= 1s)

?. TBD: Description, Title, Tags

?. TBD / IDK: thumbnails

In [2]:
# Load .csv file
gaming = pd.read_csv("/work/random_50_gaming_usa_videos.csv")
print(gaming.info())

/tmp/ipykernel_1285/3899781735.py:2: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  gaming = pd.read_csv("/work/random_50_gaming_usa_videos.csv")
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109492 entries, 0 to 109491
Data columns (total 51 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   video_id                    109492 non-null  object 
 1   publishedAt                 109492 non-null  object 
 2   channel_id                  109492 non-null  object 
 3   title                       109492 non-null  object 
 4   description                 101133 non-null  object 
 5   thumbnail_url               109492 non-null  object 
 6   thumbnail_width             109492 non-null  int64  
 7   thumbnail_height            109492 non-null  int64  
 8   channel_name                109492 non-null  object 
 9   tags                        93677 non-null  

## Dropping Columns

In [8]:
# drop useless columns

keep_cols = ["video_id", "publishedAt", "channel_id", "title", "description", "tags", "duration", "view", "like", "comment", "has_paid_product_placement", "sponsored", "collection_time", "short", "description_channel", "country"]
g2 = gaming[keep_cols] # change to in-place afterwards
print(g2.info())



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109492 entries, 0 to 109491
Data columns (total 16 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   video_id                    109492 non-null  object 
 1   publishedAt                 109492 non-null  object 
 2   channel_id                  109492 non-null  object 
 3   title                       109492 non-null  object 
 4   description                 101133 non-null  object 
 5   tags                        93677 non-null   object 
 6   duration                    109492 non-null  object 
 7   view                        109488 non-null  float64
 8   like                        107010 non-null  float64
 9   comment                     109214 non-null  float64
 10  has_paid_product_placement  109492 non-null  bool   
 11  sponsored                   109492 non-null  bool   
 12  collection_time             109492 non-null  object 
 13  short         

In [10]:
# Filter out videos not older than 3 months
g2['publishedAt'] = pd.to_datetime(g2['publishedAt'])
g2['collection_time'] = pd.to_datetime(g2['collection_time'])
g2['month_difference'] = (
    (g2['collection_time'].dt.year - g2['publishedAt'].dt.year) * 12 +
    (g2['collection_time'].dt.month - g2['publishedAt'].dt.month)
)
g2 = g2[g2['month_difference'] >= 3]
g2 = g2.drop(columns=['month_difference'])
# print(g2.info())

# Convert duration to timedelta in seconds
duration = pd.to_timedelta(g2["duration"])
g2["duration"] = duration.dt.total_seconds() / 60
# print(g2.info())


# Filter out videos not based in country of US
g2 = g2[g2['country'].eq('US')]
g2 = g2.drop(columns=['country'])
# print(g2.info())


# Filter out shorts
g2 = g2.loc[~g2['short']].copy()
g2 = g2.drop(columns=['short'])

# filter out private/archived videos
comment_errors_df = pd.read_csv("gaming_comments_errors.csv")
comment_errors_df = comment_errors_df[
    comment_errors_df['error_reason'] == 'videoNotFound'
]
g2 = g2[~g2['video_id'].isin(comment_errors_df['video_id'])]

# Filter out livestreams
g2 = g2[g2['duration'] > 1]

# Combine sponsored with has_paid_product_placement
g2['sponsored'] = g2['sponsored'] | g2['has_paid_product_placement']
g2 = g2.drop(columns=['has_paid_product_placement'])

# Filling in na values 
g2.loc[g2['description'].isna(), 'description'] = ""
g2.loc[g2['tags'].isna(), 'tags'] = "[]"
print(g2.info())

g2.to_csv('cleaned_data.csv', index=False)  


/tmp/ipykernel_1285/2883147507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g2['publishedAt'] = pd.to_datetime(g2['publishedAt'])
/tmp/ipykernel_1285/2883147507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  g2['collection_time'] = pd.to_datetime(g2['collection_time'])
/tmp/ipykernel_1285/2883147507.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

# Constructing Data

In [28]:
df = pd.read_csv('cleaned_data.csv')

In [5]:
# converting tags to an easy to use list string
def converting_tags_to_string(text):
    text = ast.literal_eval(text)
    return "|||".join(text)
df['tags'] = df['tags'].apply(converting_tags_to_string)

# generate likes-to-views ratio
g2['likes_to_views'] = g2['like'] / g2['view']

In [ ]:
# Average engegement over the last 20 videos for each channel
g2 = g2.sort_values(by=['channel_id', 'publishedAt'], ascending=True)
g2['avg_engagement_last_20'] = g2.groupby('channel_id')['engagement']
.transform(lambda x: x.rolling(window=20, min_periods=1).mean())

## Kaiden Cleaning Data

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import ast
import re
import isodate

In [7]:
gaming = pd.read_csv("random_50_gaming_usa_videos.csv")
technology = pd.read_csv("random_50_tech_usa_videos.csv")

/tmp/ipykernel_40/3090614891.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  gaming = pd.read_csv("random_50_gaming_usa_videos.csv")


FileNotFoundError: [Errno 2] No such file or directory: 'random_50_tech_usa_videos.csv'

In [5]:
df = pd.read_csv("random_50_gaming_usa_videos.csv")
std_scaler = StandardScaler()
columns = ['country','video_id','publishedAt','title','description','tags','duration','definition'
,'caption','view','like','collection_time','short','subscriber_count','comment','video_count','branding_keywords']
kdata = df[columns]
comment_errors_df = pd.read_csv("gaming_comments_errors.csv")
comment_errors_df = comment_errors_df[
    comment_errors_df['error_reason'] == 'videoNotFound'
]
kdata = kdata[kdata['country'] =='US']
kdata.drop(columns=['country'],inplace=True)
kdata = kdata[~kdata['short']]
kdata.drop(columns=['short'],inplace=True)
# filter videos with with 'live_status' to none and remove ( optional really, I don't know what it meant by upcoming and their was only like 53 of them so I removed it)
# setting null description, branding_keywords, and tags to empty strings (so they dont cause error in our model, every tag entry contains brackets)
g2.loc[g2['description'].isna(), 'description'] = ""
g2.loc[g2['branding_keywords'].isna(), 'branding_keywords'] = ""
g2.loc[g2['tags'].isna(), 'tags'] = "[]"
# converting tags and branding_keywords to string list sep by ||| (so analysis can be repetitive with tags, I chose ||| bc tags contains commas)
def converting_tags_to_string(text):
    text = ast.literal_eval(text)
    return "|||".join(text)
kdata['tags'] = kdata['tags'].apply(converting_tags_to_string)
def converting_branding_keywords_to_string (text):
    lst = re.findall(r'"([^"]+)"|\S+', text)
    return "|||".join(lst)
kdata['branding_keywords'] = kdata['branding_keywords'].apply(converting_branding_keywords_to_string)
# filter videos that are atleast 3 months old from collection time (to remove videos with potential botting)
kdata['publishedAt'] = pd.to_datetime(kdata['publishedAt'])
kdata['collection_time'] = pd.to_datetime(kdata['collection_time'])
kdata['month_diff'] = (kdata.loc[:,'collection_time'].dt.year - kdata.loc[:,'publishedAt'].dt.year) * 12 + \
                      (kdata.loc[:,'collection_time'].dt.month - kdata.loc[:,'publishedAt'].dt.month)
kdata = kdata[kdata['month_diff'] > 4]
kdata.shape
# drop columns month_diff, publishedAt, collection_time (uneccesary columns in analysis)
kdata.drop(['month_diff', 'publishedAt', 'collection_time'],inplace = True, axis = 1)
# make column engagement like/views
kdata['engagement'] = kdata['like']/kdata['view']
kdata.drop(['like','view'],inplace = True, axis = 1)
# convert definition to true or false (converting string to numeric)
kdata['hd_definition'] = kdata['definition'] == 'hd'
kdata.drop(['definition'],inplace = True, axis = 1)
# get the length for title, description, tags, and branding_keywords (since sentimental analysis wont be focusing at the length of the text)
kdata['title_len'] = kdata['title'].apply(len)
kdata['description_len'] = kdata['description'].apply(len)
kdata['num_branding_keywords'] = kdata['branding_keywords'].str.split('|||').apply(len)
kdata['num_tags'] = kdata['tags'].str.split('|||').apply(len)
# convert duration to minutes (converting string to numeric)
def minute_duration(duration):
    duration = isodate.parse_duration(duration)
    return duration.total_seconds() / 60
kdata['minute_duration'] = kdata['duration'].apply(minute_duration)
kdata.drop(['duration'], axis = 1,inplace= True)
# normalize all numerical values (to make sure weights are balanced using a minmax scalar)
numeric_columns = ['title_len','description_len','num_tags','minute_duration','subscriber_count','video_count','num_branding_keywords']
for column in numeric_columns:
    kdata[column] = std_scaler.fit_transform(kdata[[column]])



kdata.info()

/tmp/ipykernel_40/2620627726.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("random_50_gaming_usa_videos.csv")
<class 'pandas.core.frame.DataFrame'>
Index: 83283 entries, 0 to 109490
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   video_id               83283 non-null  object 
 1   title                  83283 non-null  object 
 2   description            83283 non-null  object 
 3   tags                   83283 non-null  object 
 4   caption                83283 non-null  bool   
 5   subscriber_count       83283 non-null  float64
 6   comment                83134 non-null  float64
 7   video_count            83283 non-null  float64
 8   branding_keywords      83283 non-null  object 
 9   engagement             82844 non-null  float64
 10  hd_definition          83283 non-null  bool   
 11  title_len              8

In [17]:
kdata.shape

(103488, 17)

In [9]:
import pandas as pd
data = pd.read_csv('random_50_gaming_usa_videos.csv')
data.head(5)


/tmp/ipykernel_40/4155281984.py:7: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('random_50_gaming_usa_videos.csv')


,video_id,publishedAt,channel_id,title,description,thumbnail_url,thumbnail_width,thumbnail_height,channel_name,tags,...,googlePlusUserId,view_count,subscriber_count,hidden_subscriber_count,video_count,branding_title,branding_description,branding_keywords,rank,upload_playlist
0,2HjctXU6N5I,2021-06-10T14:13:24Z,UCFl1mGlf5j0Qno1Kxnyv6FA,Why Time is the Most Valuable Resource in Leag...,Why Time is the Most Valuable Resource in Leag...,https://i.ytimg.com/vi/2HjctXU6N5I/hqdefault.jpg,480,360,Skill Capped Challenger LoL Guides,"['league of legends', 'league of legends pro g...",...,NaN,414711320,1000000,False,1526,Skill Capped Challenger LoL Guides,Skill Capped Challenger LoL Guides,"""Skill Capped"" Skillcapped Skill-Capped ""Leagu...",189,UUFl1mGlf5j0Qno1Kxnyv6FA
1,Yp-hUcvXnes,2021-09-29T10:59:50Z,UCFl1mGlf5j0Qno1Kxnyv6FA,3 BEST Champions to ESCAPE LOW ELO for END of ...,3 BEST CHAMPIONS to ESCAPE LOW ELO for EVERY R...,https://i.ytimg.com/vi/Yp-hUcvXnes/hqdefault.jpg,480,360,Skill Capped Challenger LoL Guides,"['best champions', 'best champs end of season'...",...,NaN,414711320,1000000,False,1526,Skill Capped Challenger LoL Guides,Skill Capped Challenger LoL Guides,"""Skill Capped"" Skillcapped Skill-Capped ""Leagu...",189,UUFl1mGlf5j0Qno1Kxnyv6FA
2,N5-AMeWwxqg,2019-09-06T14:30:02Z,UCFl1mGlf5j0Qno1Kxnyv6FA,How Does G2 Wunder Make Challengers Look BRONZ...,http://www.skill-capped.com/lol\nSubscribe: ht...,https://i.ytimg.com/vi/N5-AMeWwxqg/hqdefault.jpg,480,360,Skill Capped Challenger LoL Guides,"['G2 esports', 'G2', 'G2 Wunder', 'Wunder Guid...",...,NaN,414711320,1000000,False,1526,Skill Capped Challenger LoL Guides,Skill Capped Challenger LoL Guides,"""Skill Capped"" Skillcapped Skill-Capped ""Leagu...",189,UUFl1mGlf5j0Qno1Kxnyv6FA
3,-OzarB6x0q4,2021-12-18T14:00:01Z,UCFl1mGlf5j0Qno1Kxnyv6FA,How to Play Like a YASUO MAIN! - ULTIMATE YASU...,►Get the RANK You’ve Always Wanted: https://ww...,https://i.ytimg.com/vi/-OzarB6x0q4/hqdefault.jpg,480,360,Skill Capped Challenger LoL Guides,"['yasuo guide', 'yasuo', 'yasuo runes', 'yasuo...",...,NaN,414711320,1000000,False,1526,Skill Capped Challenger LoL Guides,Skill Capped Challenger LoL Guides,"""Skill Capped"" Skillcapped Skill-Capped ""Leagu...",189,UUFl1mGlf5j0Qno1Kxnyv6FA
4,2v3C7y5xu84,2024-09-15T11:30:01Z,UCFl1mGlf5j0Qno1Kxnyv6FA,Faker's CLEANEST AHRI Play of ALL TIME! #leagu...,►𝐖𝐄𝐁𝐒𝐈𝐓𝐄: https://www.skill-capped.com/lol/pri...,https://i.ytimg.com/vi/2v3C7y5xu84/hqdefault.jpg,480,360,Skill Capped Challenger LoL Guides,"['league of legends', 'lol season 14', 'lol gu...",...,NaN,414711320,1000000,False,1526,Skill Capped Challenger LoL Guides,Skill Capped Challenger LoL Guides,"""Skill Capped"" Skillcapped Skill-Capped ""Leagu...",189,UUFl1mGlf5j0Qno1Kxnyv6FA


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=395a814d-9932-4e75-96c1-b9471a984f5a' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>